In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from multiprocessing import Pool

import numpy as np
import pandas as pd

import kraft

In [ ]:
SETTING = kraft.json.read("setting.json")

In [ ]:
directory_path = "{}/gene_set/".format(SETTING["directory_path"])

kraft.path.path(directory_path)

In [ ]:
gene_x_sample = pd.read_csv(
    "{}/../rna/gene_x_cell_line.tsv".format(directory_path), sep="\t", index_col=0
)

gene_x_sample = gene_x_sample.loc[
    gene_x_sample.index & kraft.name_biology.select_genes(), :
]

In [ ]:
gene_set_to_genes = kraft.gmt.read_many(
    tuple(
        "data/gene_set/{}".format(file_name)
        for file_name in (
            "h.all.v7.1.symbols.gmt",
            "c1.all.v7.1.symbols.gmt",
            "c2.all.v7.1.symbols.gmt",
            "c3.all.v7.1.symbols.gmt",
            "c5.all.v7.1.symbols.gmt",
            "c6.all.v7.1.symbols.gmt",
            "c7.all.v7.1.symbols.gmt",
            "ipa.gmt",
        )
    )
)

In [ ]:
# TODO: add to kraft
def function(scores, set_to_elements):

    scores = pd.Series(
        kraft.array.normalize(scores.to_numpy(), "-0-"),
        index=scores.index,
        name=scores.name,
    ).sort_values()

    return tuple(
        kraft.sea.score_set(
            scores, elements, method=method, sort_element_score=False, plot=False
        )
        for elements in set_to_elements.values()
    )

In [ ]:
method = "classic"

In [ ]:
pool = Pool(processes=8)

gene_set_x_sample = pd.DataFrame(
    np.asarray(
        pool.starmap(
            function,
            ((scores, gene_set_to_genes) for _, scores in gene_x_sample.items()),
        )
    ).T,
    index=gene_set_to_genes.keys(),
    columns=gene_x_sample.columns,
)

gene_set_x_sample.to_csv(
    "{}/gene_set_x_sample_x_{}.tsv".format(directory_path, method), sep="\t"
)

gene_set_x_sample